In [2]:
!pip install nltk

     ---------------------------------------- 0.0/42.0 kB ? eta -:--:--
     ---------------------------------------- 0.0/42.0 kB ? eta -:--:--
     ---------------------------------------- 0.0/42.0 kB ? eta -:--:--
     --------- ------------------------------ 10.2/42.0 kB ? eta -:--:--
     --------------------------- ---------- 30.7/42.0 kB 435.7 kB/s eta 0:00:01
     -------------------------------------- 42.0/42.0 kB 337.6 kB/s eta 0:00:00
  Using cached tqdm-4.66.4-py3-none-any.whl.metadata (57 kB)
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
    --------------------------------------- 0.0/1.5 MB 435.7 kB/s eta 0:00:04
   - -------------------------------------- 0.0/1.5 MB 393.8 kB/s eta 0:00:04
   - -------------------------------------- 0.1/1.5 MB 409.6 kB/s eta 0:00:04
   - -------------------------------------- 0.1/1.5 MB 409.6 kB/s eta 0:00:04
   - -----------------------------------


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
import random
import pickle
import heapq

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nltk.tokenize import RegexpTokenizer

import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Activation
from tensorflow.keras.optimizers import RMSprop

In [3]:
text_df = pd.read_csv("fake_or_real_news.csv")
subset_text_df = text_df.sample(frac=0.001, random_state=42)  # Adjust the fraction as needed
text = list(subset_text_df.text.values)
joined_text = " ".join(subset_text_df.text.values)

with open("joined_text.txt", "w", encoding="utf-8") as f:
    f.write(joined_text)

In [12]:
text_df.shape

(6335, 4)

In [21]:
text_df.head(10)

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL
5,6903,"Tehran, USA","\nI’m not an immigrant, but my grandparents ...",FAKE
6,7341,Girl Horrified At What She Watches Boyfriend D...,"Share This Baylee Luciani (left), Screenshot o...",FAKE
7,95,‘Britain’s Schindler’ Dies at 106,A Czech stockbroker who saved more than 650 Je...,REAL
8,4869,Fact check: Trump and Clinton at the 'commande...,Hillary Clinton and Donald Trump made some ina...,REAL
9,2909,Iran reportedly makes new push for uranium con...,Iranian negotiators reportedly have made a las...,REAL


In [24]:
students = ["Ida", "Anne"]
for position,student in enumerate(students):
    print(student)
    print(position)
ds = []
for i in range(len(students)):
    ds.append(students[i:i + 5])
print(ds)

Ida
0
Anne
1
[['Ida', 'Anne'], ['Anne']]


In [26]:
for position,student in enumerate(students):
    for index, character in enumerate(student):
        print(index)
        print(character)
    print(position)
    print(student)

0
I
1
d
2
a
0
Ida
0
A
1
n
2
n
3
e
1
Anne


In [28]:
partial_text = joined_text[:1000000]

In [29]:
tokenizer = RegexpTokenizer(r"\w+")
tokens = tokenizer.tokenize(partial_text.lower())

In [30]:
unique_tokens = np.unique(tokens)
unique_token_index = {token: index for index, token in enumerate(unique_tokens)}

In [31]:
n_words = 10
input_words = []
next_word = []

for i in range(len(tokens) - n_words):
    input_words.append(tokens[i:i + n_words])
    next_word.append(tokens[i + n_words])

In [33]:
X = np.zeros((len(input_words), n_words, len(unique_tokens)), dtype=bool)  # for each sample, n input words and then a boolean for each possible next word
y = np.zeros((len(next_word), len(unique_tokens)), dtype=bool)  # for each sample a boolean for each possible next word


In [36]:
for i, words in enumerate(input_words):
    for j, word in enumerate(words):
        X[i, j, unique_token_index[word]] = 1
    y[i, unique_token_index[next_word[i]]] = 1


In [9]:
model = Sequential()
model.add(LSTM(128, input_shape=(n_words, len(unique_tokens)), return_sequences=True))
model.add(LSTM(128))
model.add(Dense(len(unique_tokens)))
model.add(Activation("softmax"))

In [10]:
optimizer = RMSprop(learning_rate=0.01)
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
history = model.fit(X, y, batch_size=128, epochs=10, shuffle=True).history

Epoch 1/10


54/54 [==============================] - 8s 74ms/step - loss: 6.7207 - accuracy: 0.0545
Epoch 2/10
54/54 [==============================] - 4s 82ms/step - loss: 6.4782 - accuracy: 0.0599
Epoch 3/10
54/54 [==============================] - 4s 79ms/step - loss: 6.3959 - accuracy: 0.0698
Epoch 4/10
54/54 [==============================] - 4s 80ms/step - loss: 6.2955 - accuracy: 0.0757
Epoch 5/10
54/54 [==============================] - 5s 89ms/step - loss: 6.1325 - accuracy: 0.0854
Epoch 6/10
54/54 [==============================] - 5s 90ms/step - loss: 5.9593 - accuracy: 0.0900
Epoch 7/10
54/54 [==============================] - 4s 81ms/step - loss: 5.7533 - accuracy: 0.0978
Epoch 8/10
54/54 [==============================] - 5s 92ms/step - loss: 5.5578 - accuracy: 0.1053
Epoch 9/10
54/54 [==============================] - 5s 93ms/step - loss: 5.3488 - accuracy: 0.1118
Epoch 10/10
54/54 [==============================] - 5s 91ms/step - loss: 5.1097 - accuracy: 0.1298


In [11]:
history = model.fit(X, y, batch_size=128, epochs=5, shuffle=True).history

Epoch 1/5
54/54 [==============================] - 5s 85ms/step - loss: 4.8376 - accuracy: 0.1526
Epoch 2/5
54/54 [==============================] - 4s 81ms/step - loss: 4.5520 - accuracy: 0.1786
Epoch 3/5
54/54 [==============================] - 4s 81ms/step - loss: 4.2457 - accuracy: 0.2101
Epoch 4/5
54/54 [==============================] - 4s 78ms/step - loss: 3.9318 - accuracy: 0.2401
Epoch 5/5
54/54 [==============================] - 4s 73ms/step - loss: 3.6150 - accuracy: 0.2862


In [12]:
model.save("text_gen_model2.h5")
with open("history2.p", "wb") as f:
    pickle.dump(history, f)

C:\Users\USER\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [13]:
model = load_model("text_gen_model2.h5")
history = pickle.load(open("history2.p", "rb"))

In [14]:
def predict_next_word(input_text, n_best):
    input_text = input_text.lower()
    X = np.zeros((1, n_words, len(unique_tokens)))
    for i, word in enumerate(input_text.split()):
        X[0, i, unique_token_index[word]] = 1
        
    predictions = model.predict(X)[0]
    return np.argpartition(predictions, -n_best)[-n_best:]

In [15]:
possible = predict_next_word("I will have to look into this thing because I", 5)

1/1 [==============================] - 1s 840ms/step


In [16]:
for idx in possible:
    print(unique_tokens[idx])

were
now
their
because
it


In [17]:
def generate_text(input_text, n_words, creativity=3):
    word_sequence = input_text.split()
    current = 0
    for _ in range(n_words):
        sub_sequence = " ".join(tokenizer.tokenize(" ".join(word_sequence).lower())[current:current+n_words])
        try:
            choice = unique_tokens[random.choice(predict_next_word(sub_sequence, creativity))]
        except:
            choice = random.choice(unique_tokens)
            print("Sorry, I didn't get that quiet well. Maybe you can contact our sales representative at someone'sname_sales@citybi.co.ke")
        word_sequence.append(choice)
        current += 1
    return " ".join(word_sequence)

In [18]:
generate_text("I will have to look into this thing because I", 100, 10)

1/1 [==============================] - 0s 26ms/step


'I will have to look into this thing because I own such there it it if to also by that to what about them there there the no market by an week there in a form with syria agents comey lives to be lives as said her have on is be for for its than on under clear is that is no only for like and all but it on what they were there by well the u s u but aristocracy t any controlled and the u of these other dunk having or not of an event on he are he are are history to his former about and'

In [19]:
generate_text("The president of the United States announced yesterday that he", 100, 10)

1/1 [==============================] - 0s 22ms/step


'The president of the United States announced yesterday that he acts called federal establishment most t occasion 2008 it for it or get media america as be back but clinton or able for if one such that they might they have not as a federal market for the black market also an a market for information it that a no alter also against has than as lives and kill clinton after thus trump or well we been reported for an been between is to hillary a fbi up would simply to all that that have no been trump say to much but a no black against with there to be'

In [20]:
for idx in predict_next_word("The president will most likely not be there to ", 5):
    print(unique_tokens[idx])

1/1 [==============================] - 0s 23ms/step
she
an
the
no
a


In [24]:
generate_text("who is the kenyan president ", 12, 5)

1/1 [==============================] - 0s 25ms/step


'who is the kenyan president strictly 2 put urging are there they are is they a have'